In [ ]:
from ultralytics import YOLO

IMG_DATA = "/kaggle/input/large-license-plate-dataset/images/{}/"
LABEL_DATA = "/kaggle/input/large-license-plate-dataset/labels/{}/"

model = YOLO('yolov8n.pt')

model.train(
    data='/kaggle/input/cv-project-files2/dataset.yaml',
    epochs=75,                            # Number of training epochs
    imgsz=640,                            # Image size
    batch=50,                             # batch size
    lr0=0.01,                             # initial learning rate
    augment=False,                         # no data augmentation
    dropout=0.2       ,                    # dropout to reduce overfitting
    device=[0,1]
)


In [ ]:
!pip install ultralytics
!pip install -U ipywidgets

In [ ]:
import os
import cv2

image_folder = '/kaggle/input/large-license-plate-dataset/images/test'
model = YOLO('/kaggle/working/runs/detect/train2/weights/best.pt') 

for image_name in os.listdir(image_folder):
    if image_name.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)

        results = model(image)

        if isinstance(results, list):
            for result in results:
                output_image_path = os.path.join('/kaggle/working/test', f'detected_{image_name}')
                result.show() 
                result.save(output_image_path)
        else:
            results.show()
